In [77]:
from pykrx import stock
import pandas as pd
import numpy as np
import os
import time

In [78]:
# 백테스트 할 날짜를 생성하기
def make_date_list(m, d):
    date_list = []
    for y in range(2002, 2022): # 백테스트 기간
        day = str(y) + str(m) + str(d)
        b_day = stock.get_nearest_business_day_in_a_week(date=day) #휴일이면 근처의 영업일로 변경
        date_list.append(b_day) 
    return date_list

In [79]:
date = make_date_list(6,30)

In [80]:
# 연도별 수익률 구하기
def row_pbr_dps(date, date2, a, b): #PBR과 DPS 기반 전략

    # 종목코드와 종목명으로 데이터프레임 만들기 
    codes = stock.get_market_ticker_list(date, market='ALL') # 코스피, 코스닥 데이터의 종목코드 데이터
    corp = []
    for code in codes:
        name = stock.get_market_ticker_name(code) # 코스피, 코스닥 데이터의 종목명 데이터
        corp.append([code, name])
    df1 = pd.DataFrame(data=corp, columns=['code', '종목명']) 
    df1 = df1.set_index('code') # 종목코드 인덱스화 
 
    # 종목 펀더멘털 데이터와 종가, 1년 후 종가 불러오기 
    df_f = stock.get_market_fundamental_by_ticker(date=date, market='ALL')  # PER, PBR, DIV,... : KRX 우선가져오기
    df_c = stock.get_market_cap_by_ticker(date=date, market='ALL')  # 종가 
    df_c2 = stock.get_market_cap_by_ticker(date2, market='ALL')  # 1년 후 종가 
 
    time.sleep(1) # 프로그램 1초 정지
    
    df_c2 = df_c2[['종가', '상장주식수']]
    
    df = pd.merge(df1, df_c, left_index=True, right_index=True)
    df = pd.merge(df, df_f, left_index=True, right_index=True)
    df = pd.merge(df, df_c2, left_index=True, right_index=True)
 
    df = df[['종목명', '종가_x', '상장주식수_x', a, b, '종가_y', '상장주식수_y', '거래량']]
    df.columns = ['종목명', '종가', '상장주식수', a, b, '1년후종가', '1년후상장주식수', '거래량']
    df['상장주식수변동'] = df['1년후상장주식수'] - df['상장주식수']
 
    df = df[df[a] > 0]  # PBR 0이상만 구하기
    df[a + '_rank'] = df[a].rank()
 
    df = df[df[b] > 0] # # DPS 0이상만 구하기
    df[b + '_rank'] = df[b].rank()
 
    df[a+b] = df[a + '_rank'] + df[b + '_rank']
    df[a + b + '_rank'] = df[a+b].rank()
    df = df.sort_values([a + b + '_rank'])
 
    df = df[df['거래량'] > 0]  # 거래량이 0 이상인 종목만 구하기

    df = df.iloc[:30]  # 종목개수
    df['수익'] = df['1년후종가'] - df['종가']
 
    df['수익'].loc[df['상장주식수변동'] < 0] = df['1년후종가'] * (1 + df['상장주식수변동'] / df['상장주식수']) - df['종가']
 
    df['수익률'] = (df['수익'] / df['종가'])
    df['투자년도'] = np.array([date] * len(df))
 
    p = df['수익률'].mean()
 
    result = []
    result.append([date, date2, p])
 
    df_t = pd.DataFrame(data=result, columns=['투자일', '1년후', '수익률'])
    return df, df_t

In [81]:
# 투자 시작년도부터 마직막 년도까지 반복
def inverst_years(date_list): # 맨 위에서 만든 백테스트 투자일 리스트
    for n in range(len(date_list)):
        if n < len(date_list)-1:
            date = date_list[n]
            date2 = date_list[n+1]

            if n == 0:
                df_t = row_pbr_dps(date, date2, 'PER', 'PBR')[1]
                df = row_pbr_dps(date, date2, 'PER', 'PBR')[0]
 
            else:
                df_t = pd.concat([df_t, row_pbr_dps(date, date2, 'PER', 'PBR')[1]])
                df = pd.concat([df, row_pbr_dps(date, date2, 'PER', 'PBR')[0]])
    return df_t

In [83]:
a = inverst_years(date)

C:\Users\0327c\AppData\Local\Temp\ipykernel_7400\2273520826.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['수익'].loc[df['상장주식수변동'] < 0] = df['1년후종가'] * (1 + df['상장주식수변동'] / df['상장주식수']) - df['종가']
C:\Users\0327c\AppData\Local\Temp\ipykernel_7400\2273520826.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['수익'].loc[df['상장주식수변동'] < 0] = df['1년후종가'] * (1 + df['상장주식수변동'] / df['상장주식수']) - df['종가']
C:\Users\0327c\AppData\Local\Temp\ipykernel_7400\2273520826.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

In [95]:
def cagr(m):
    원금 = 100000000
    for i in range(len(m)):
        원금 += 원금 * m.iloc[i, 2]
    수익률 = (원금 - 100000000) / 100000000
    return 원금, 수익률

cagr(a)

(2646157728.6753454, 2546.1577286753454)

In [ ]:
test_days = ['4', '27'] # 매월 매수/매도일
 
for d in test_days:
    origin_path = 'C:\수익왕 김퀀트'
    folder_name = '\\PBR_DPS백테스트(매월' + d + '일)거래량 0이상(30개)_All Market'
    os.mkdir(origin_path + folder_name)
 
    # 월별로 테스트를 반복하여 결과 얻기
    for m in range(12):
        if m == 0:
            m = '01'
            # 연도별 날짜 List 만들기
            date_list = make_date_list(m, d)
 
            # 백테스트 구동하기
            df_t = inverst_years(date_list)
 
        else:
            m += 1
            m = '0' + str(m)
            m = m[-2:]
            print(m)
 
            #연도별 날짜 List 만들기
            date_list = make_date_list(m, d)
 
            #백테스트 구동하기
            df_t = pd.concat([df_t, inverst_years(date_list)])